In [51]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Path to the folder containing the .csv files
folder_path = 'data/csv-clean-renamed'  # Adjust the path to the directory

# Read the data
wavenumbers = []
data = []
labels = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        filepath = os.path.join(folder_path, filename)
        spectrum = pd.read_csv(filepath, delimiter=',', header=None)
        wavenumbers.append(spectrum[0].values)  # Assuming first column is wavelength
        data.append(spectrum[1].values)  # Assuming second column is intensity
        # Extract concentration from filename
        concentration = float(filename.split('_')[0].replace('conc-', ''))
        labels.append(concentration)

wavenumbers = np.array(wavenumbers)
data = np.array(data)
labels = np.array(labels)

max_wavenumber = 1800
index_max_1500 = np.where(wavenumbers[0] <= max_wavenumber)[0][-1]
data_trimmed = data[:, :index_max_1500 + 1]
data_normalized = np.array([spectrum / np.sum(spectrum) for spectrum in data_trimmed])

# Standard scaling
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_normalized)

X = data_normalized
y = labels

# Define PyTorch dataset and dataloader
class SpectraDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define the neural network model
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Training function
def train_model(model, criterion, optimizer, dataloader, num_epochs=200):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            predictions.append(outputs.numpy())
            true_labels.append(labels.numpy())
    return np.vstack(predictions), np.vstack(true_labels)

# Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores, mae_scores, r2_scores = [], [], []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_dataset = SpectraDataset(X_train, y_train)
    test_dataset = SpectraDataset(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = Net(input_size=X_train.shape[1])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_model(model, criterion, optimizer, train_loader, num_epochs=250)
    predictions, true_labels = evaluate_model(model, test_loader)

    mse = mean_squared_error(true_labels, predictions)
    mae = mean_absolute_error(true_labels, predictions)
    r2 = r2_score(true_labels, predictions)

    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)

print(f"Cross-Validated MSE: {np.mean(mse_scores)} ± {np.std(mse_scores)}")
print(f"Cross-Validated MAE: {np.mean(mae_scores)} ± {np.std(mae_scores)}")
print(f"Cross-Validated R^2: {np.mean(r2_scores)} ± {np.std(r2_scores)}")

# Feature importance
weights = model.fc1.weight.data.numpy()
feature_importance = np.sum(np.abs(weights), axis=0)
indices = np.argsort(feature_importance)[::-1]

# print("Feature ranking:")
# for i in range(len(feature_importance)):
#     print(f"Feature {indices[i]} (Wavenumber {wavenumbers[0][indices[i]]}): {feature_importance[indices[i]]}")

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Final evaluation on a separate test set
model.eval()
predictions = model(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
r2 = r2_score(y_test, predictions)
print(f"R^2 on test set: {r2}")

# Dictionary containing true values for the unknown samples
true_values = {
    'test-file-20.dpt': 20.0,  # Replace with actual values
    'test-file-66.dpt': 66.6,
    'test-file-33.dpt': 33.3,
    'test-file-50.dpt': 50.0,
    'test-file-51.dpt': 50.0
}

# Prediction for an unknown sample
def predict_concentration_for_unknown_samples(unknown_sample_filename):
    print(f'File name: {unknown_sample_filename}')
    unknown_sample_filepath = os.path.join('C:\\Users\\baubl\\OneDrive\\Stalinis kompiuteris\\Nuova cartella\\test', unknown_sample_filename)
    unknown_spectrum = pd.read_csv(unknown_sample_filepath, delimiter=',', header=None)
    index_max_1800 = np.where(unknown_spectrum[0] <= max_wavenumber)[0][-1]
    data_trimmed_unknown = unknown_spectrum[1].values[:index_max_1800 + 1]
    normalized_unknown_spectrum = data_trimmed_unknown / np.sum(data_trimmed_unknown)
    normalized_unknown_spectrum = scaler.transform([normalized_unknown_spectrum])  # Apply same scaling
    normalized_unknown_spectrum = torch.tensor(normalized_unknown_spectrum, dtype=torch.float32)

    predicted_percentage = model(normalized_unknown_spectrum).item()
    print(f'Predicted Percentage: {predicted_percentage}')
    
    # Calculate MSE and MAE for the prediction
    true_value = true_values[unknown_sample_filename]
    mse_value = mean_squared_error([true_value], [predicted_percentage])
    mae_value = mean_absolute_error([true_value], [predicted_percentage])
    print(f'MSE for {unknown_sample_filename}: {mse_value}')
    print(f'MAE for {unknown_sample_filename}: {mae_value}')

# Predict and print MSE and MAE values for each file
predict_concentration_for_unknown_samples('test-file-20.dpt')
predict_concentration_for_unknown_samples('test-file-66.dpt')
predict_concentration_for_unknown_samples('test-file-33.dpt')
predict_concentration_for_unknown_samples('test-file-50.dpt')
predict_concentration_for_unknown_samples('test-file-51.dpt')

# Feature importance
weights = model.fc1.weight.data.numpy()
feature_importance = np.sum(np.abs(weights), axis=0)
indices = np.argsort(feature_importance)[::-1]







 


Cross-Validated MSE: 62.5645751953125 ± 42.22898864746094
Cross-Validated MAE: 5.701310634613037 ± 2.4557509422302246
Cross-Validated R^2: 0.9307197451591491 ± 0.04716478768497243
R^2 on test set: 0.9314345549773472
File name: test-file-20.dpt
Predicted Percentage: 22.097980499267578
MSE for test-file-20.dpt: 4.401522175307036
MAE for test-file-20.dpt: 2.097980499267578
File name: test-file-66.dpt
Predicted Percentage: 59.00474166870117
MSE for test-file-66.dpt: 57.68794911916417
MAE for test-file-66.dpt: 7.595258331298822
File name: test-file-33.dpt
Predicted Percentage: 40.44773864746094
MSE for test-file-33.dpt: 51.09016777240675
MAE for test-file-33.dpt: 7.14773864746094
File name: test-file-50.dpt
Predicted Percentage: 46.09074783325195
MSE for test-file-50.dpt: 15.2822525032243
MAE for test-file-50.dpt: 3.909252166748047
File name: test-file-51.dpt
Predicted Percentage: 52.104339599609375
MSE for test-file-51.dpt: 4.428245150484145
MAE for test-file-51.dpt: 2.104339599609375


In [55]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Path to the folder containing the .csv files
folder_path = 'data/csv-clean-renamed'  # Adjust the path to the directory

# Read the data
wavenumbers = []
data = []
labels = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        filepath = os.path.join(folder_path, filename)
        spectrum = pd.read_csv(filepath, delimiter=',', header=None)
        wavenumbers.append(spectrum[0].values)  # Assuming first column is wavelength
        data.append(spectrum[1].values)  # Assuming second column is intensity
        # Extract concentration from filename
        concentration = float(filename.split('_')[0].replace('conc-', ''))
        labels.append(concentration)

wavenumbers = np.array(wavenumbers)
data = np.array(data)
labels = np.array(labels)

max_wavenumber = 1800
index_max_1500 = np.where(wavenumbers[0] <= max_wavenumber)[0][-1]
data_trimmed = data[:, :index_max_1500 + 1]
data_normalized = np.array([spectrum / np.sum(spectrum) for spectrum in data_trimmed])

# Standard scaling
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_normalized)

X = data_normalized
y = labels

# Define PyTorch dataset and dataloader
class SpectraDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define the neural network model
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Training function
def train_model(model, criterion, optimizer, dataloader, num_epochs=200):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            predictions.append(outputs.numpy())
            true_labels.append(labels.numpy())
    return np.vstack(predictions), np.vstack(true_labels)

# Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores, mae_scores, r2_scores = [], [], []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_dataset = SpectraDataset(X_train, y_train)
    test_dataset = SpectraDataset(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = Net(input_size=X_train.shape[1])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_model(model, criterion, optimizer, train_loader, num_epochs=250)
    predictions, true_labels = evaluate_model(model, test_loader)

    mse = mean_squared_error(true_labels, predictions)
    mae = mean_absolute_error(true_labels, predictions)
    r2 = r2_score(true_labels, predictions)

    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)

print(f"Cross-Validated MSE: {np.mean(mse_scores)} ± {np.std(mse_scores)}")
print(f"Cross-Validated MAE: {np.mean(mae_scores)} ± {np.std(mae_scores)}")
print(f"Cross-Validated R^2: {np.mean(r2_scores)} ± {np.std(r2_scores)}")

# Feature importance
weights = model.fc1.weight.data.numpy()
feature_importance = np.sum(np.abs(weights), axis=0)
indices = np.argsort(feature_importance)[::-1]

# Identify Peaks
example_spectrum = np.mean(data_normalized, axis=0)  # Using the mean spectrum for peak detection
peaks, properties = find_peaks(example_spectrum, height=0.05, distance=20)  # Adjust height and distance parameters as needed)  # Adjust height parameter as needed

print(f"Detected Peaks: {peaks}")

# Compute Feature Importance for Peaks
peak_importances = []

for peak in peaks:
    # Define a range around the peak (you may need to adjust the range width)
    peak_range = range(max(0, peak-2), min(len(feature_importance), peak+3))
    
    # Compute the importance for the peak as the sum of importances within the peak range
    peak_importance = np.sum(feature_importance[list(peak_range)])
    peak_importances.append((wavenumbers[0][peak], peak_importance))

# Sort peaks by their importance
peak_importances.sort(key=lambda x: x[1], reverse=True)

# Extract wavenumbers and importances for plotting
if peak_importances:
    peak_wavenumbers, peak_importance_values = zip(*peak_importances)

    # Plot the peak importances
    plt.figure(figsize=(14, 8))
    plt.bar(range(len(peak_wavenumbers)), peak_importance_values, align='center')
    plt.xticks(range(len(peak_wavenumbers)), peak_wavenumbers, rotation=90)
    plt.xlabel('Wavenumber (Peak)')
    plt.ylabel('Peak Importance')
    plt.title('Peak Importances')
    plt.tight_layout()
    plt.show()
else:
    print("No peaks detected or processed.")

# Final evaluation on a separate test set
model.eval()
predictions = model(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
r2 = r2_score(y_test, predictions)
print(f"R^2 on test set: {r2}")

# Dictionary containing true values for the unknown samples
true_values = {
    'test-file-20.dpt': 20.0,  # Replace with actual values
    'test-file-66.dpt': 66.6,
    'test-file-33.dpt': 33.3,
    'test-file-50.dpt': 50.0,
    'test-file-51.dpt': 50.0
}

# Prediction for an unknown sample
def predict_concentration_for_unknown_samples(unknown_sample_filename):
    print(f'File name: {unknown_sample_filename}')
    unknown_sample_filepath = os.path.join('C:\\Users\\baubl\\OneDrive\\Stalinis kompiuteris\\Nuova cartella\\test', unknown_sample_filename)
    unknown_spectrum = pd.read_csv(unknown_sample_filepath, delimiter=',', header=None)
    index_max_1800 = np.where(unknown_spectrum[0] <= max_wavenumber)[0][-1]
    data_trimmed_unknown = unknown_spectrum[1].values[:index_max_1800 + 1]
    normalized_unknown_spectrum = data_trimmed_unknown / np.sum(data_trimmed_unknown)
    normalized_unknown_spectrum = scaler.transform([normalized_unknown_spectrum])  # Apply same scaling
    normalized_unknown_spectrum = torch.tensor(normalized_unknown_spectrum, dtype=torch.float32)

    predicted_percentage = model(normalized_unknown_spectrum).item()
    print(f'Predicted Percentage: {predicted_percentage}')
    
    # Calculate MSE and MAE for the prediction
    true_value = true_values[unknown_sample_filename]
    mse_value = mean_squared_error([true_value], [predicted_percentage])
    mae_value = mean_absolute_error([true_value], [predicted_percentage])
    print(f'MSE for {unknown_sample_filename}: {mse_value}')
    print(f'MAE for {unknown_sample_filename}: {mae_value}')

# Predict and print MSE and MAE values for each file
predict_concentration_for_unknown_samples('test-file-20.dpt')
predict_concentration_for_unknown_samples('test-file-66.dpt')
predict_concentration_for_unknown_samples('test-file-33.dpt')
predict_concentration_for_unknown_samples('test-file-50.dpt')
predict_concentration_for_unknown_samples('test-file-51.dpt')


Cross-Validated MSE: 40.792747497558594 ± 21.77153205871582
Cross-Validated MAE: 4.825381278991699 ± 1.829215407371521
Cross-Validated R^2: 0.955007815361023 ± 0.024078592290517502
Detected Peaks: []
No peaks detected or processed.
R^2 on test set: 0.935496237271733
File name: test-file-20.dpt
Predicted Percentage: 22.205801010131836
MSE for test-file-20.dpt: 4.865558096298628
MAE for test-file-20.dpt: 2.205801010131836
File name: test-file-66.dpt
Predicted Percentage: 67.46195220947266
MSE for test-file-66.dpt: 0.7429616114148037
MAE for test-file-66.dpt: 0.8619522094726619
File name: test-file-33.dpt
Predicted Percentage: 46.634307861328125
MSE for test-file-33.dpt: 177.8037661406771
MAE for test-file-33.dpt: 13.334307861328128
File name: test-file-50.dpt
Predicted Percentage: 50.88127136230469
MSE for test-file-50.dpt: 0.7766392140183598
MAE for test-file-50.dpt: 0.8812713623046875
File name: test-file-51.dpt
Predicted Percentage: 56.36664581298828
MSE for test-file-51.dpt: 40.53417